In [76]:
import json
#import ast
import re
import numpy as np

import time
import pandas as pd
import numpy as np
#from bs4 import BeautifulSoup
#import os

In [3]:
#filename = 'MeDOAS_Spec_20190621_103618.json'
#filename = 'MeDOAS_Spec_20190710_001316.json'
path = '.\\plcData\\'

In [4]:
# with open(path+filename, 'r') as f:
#     s = f.read()
    #d = json.load(f)

# Fallback: parse data from json string


In [125]:
# Function declarations


def ReadLinesFromFile(s): 
    bracket_depth= 0
    values = []
    newBracket = -1
    substring = ''

    read_subsequent_lines = False

    new_values =''

    for i in s.split('\n'):
        if '{' in i:
            bracket_depth+=i.count('{')
        
        if '}' in i:
            bracket_depth-=i.count('}')
            read_subsequent_lines = False
        if ':' in i:
            
            if ("PC time" in i)|("PLC time" in i):
                values.append(i.replace('"',''))
            
            if ('Temp' in i)|('Temperature' in i):
                read_subsequent_lines = True
                newBracket = bracket_depth

            if read_subsequent_lines & (bracket_depth == newBracket):
                new_values = new_values + i
            else:
                if new_values != '':
                    values.append(new_values.replace('{', '').replace('"',''))
                read_subsequent_lines = False
                new_values =''
                
#         print(bracket_depth, ', ', i)
    return values



def ProcessValues(values):
    most_recent_time = -1

    df = pd.DataFrame(columns=['Time', 'Temperature', 'Original'])

    for line in values:
        tokens = line.split(':')
        if 'time' in tokens[0]:
            most_recent_time = line.replace(tokens[0]+':','').replace(',', '').replace('USA', '')
        if ('Temp' in tokens[0])|('Temperature' in tokens[0]):
            temp = tokens[-1].strip()
            if temp.isnumeric():
                numDecimals = int(tokens[2].split(',')[0])
                temp = float(temp[0:numDecimals] + '.'+ temp[numDecimals:])
                df = df.append({'Time': most_recent_time, 'Temperature': temp, 'Original': tokens[0]}, ignore_index=True)

    return df


In [70]:
files = [x for x in os.listdir('plcData') if x.split('.')[-1] == 'json']



In [71]:
file_types = set([x.split('_')[0] for x in files])
file_types

{'MeDOAS', 'MeFTIR', 'SkyDOAS'}

In [130]:
t= time.time()

files_short = [x for x in files if 'MeDOAS' == x.split('_')[0]]
df_MeDoas = pd.DataFrame(columns=['Time', 'Temperature', 'Original'])
for filename in files_short:
    with open(path+filename, 'r') as f:
        s = f.read()
        values = ReadLinesFromFile(s)
        df = ProcessValues(values)
        df_MeDoas = df_MeDoas.append(df, ignore_index=True)

print('Finished MeDOAS')        
print(time.time()-t)
df_MeDoas.to_csv('MeDOAS.csv')


files_short = [x for x in files if 'MeFTIR' == x.split('_')[0]]
df_MeFTIR = pd.DataFrame(columns=['Time', 'Temperature', 'Original'])
for filename in files_short:
    with open(path+filename, 'r') as f:
        s = f.read()
        values = ReadLinesFromFile(s)
        df = ProcessValues(values)
        df_MeFTIR = df_MeFTIR.append(df, ignore_index=True)

print('Finished MeFTIR')        
print(time.time()-t)
df_MeFTIR.to_csv('MeFTIR.csv')

        
files_short = [x for x in files if 'SkyDOAS' == x.split('_')[0]]
df_SkyDOAS = pd.DataFrame(columns=['Time', 'Temperature', 'Original'])
for filename in files_short:
    with open(path+filename, 'r') as f:
        s = f.read()
        values = ReadLinesFromFile(s)
        df = ProcessValues(values)
        df_SkyDOAS = df_SkyDOAS.append(df, ignore_index=True)
        
print('Finished SkyDOAS')                
print(time.time()-t)
df_SkyDOAS.to_csv('SkyDOAS.csv')


Finished MeDOAS
1680.79745054245
Finished MeFTIR
2510.796069383621


NameError: name 'df_SkyDOASR' is not defined